# Quadratic Unconstrained Binary Optimization (QUBO)

Quadratic unconstrained binary optimization (QUBO) is a pattern matching technique, common in machine learning applications. QUBO is an NP hard problem. Examples of problems that can be formulated as QUBO problems are the Maximum cut, Graph coloring and the Partition problem. QUBO problems may sometimes be well-suited to algorithms aided by quantum annealing

[Examples](https://arxiv.org/ftp/arxiv/papers/1811/1811.11538.pdf)

[Dimod Documents](https://readthedocs.com/projects/d-wave-systems-dimod/downloads/pdf/latest/)

[Japan Tutorial](https://tc3-japan.github.io/DA_tutorial/tutorial-1-sudoku.html)

In [6]:
#pip install dwave-ocean-sdk

In [1]:
import numpy as np
import dimod

The QUBO form, $E(𝑎_𝑖, 𝑏_{𝑖,𝑗} ; 𝑞_{𝑖}) = −𝑞_1 − 𝑞_2 + 2𝑞_1 𝑞_2$ , is related to the Ising form, 

$E(ℎ_𝑖, 𝑗_{𝑖,𝑗}; 𝑠_𝑖) = \frac{1}{2} (𝑠_1𝑠_2 − 1)$, via

the simple manipulation $𝑠_𝑖 = 2𝑞_𝑖 − 1$.


In [2]:
bqm = dimod.BinaryQuadraticModel({0: -1, 1: -1}, {(0, 1): 2}, 0.0, dimod.BINARY)
bqm_ising = bqm.change_vartype(dimod.SPIN, inplace=False)

In [3]:
response = dimod.ExactSolver().sample(bqm)

In [4]:
for sample, energy in response.data(['sample', 'energy']):
    print(sample, energy)

{0: 1, 1: 0} -1.0
{0: 0, 1: 1} -1.0
{0: 0, 1: 0} 0.0
{0: 1, 1: 1} 0.0


### 1. Minimize

$y = -5x_1 -3x_2 - 8x_3 - 6x_4 + 4x_1x_2 + 8x_1x_3 + 2x_2x_3 + 10x_3x_4$

In [11]:
linear = {0:-5,1:-3,2:-8,3:-6}
quadratic = {(0,1):4, (0,2):8,(1,2):2,(2,3):10}

In [12]:
bqm = dimod.BinaryQuadraticModel(linear, quadratic, 0.0, dimod.BINARY)
response = dimod.ExactSolver().sample(bqm)

In [13]:
for sample, energy in response.data(['sample', 'energy']):
    print(sample, energy)

{0: 1, 1: 0, 2: 0, 3: 1} -11.0
{0: 1, 1: 1, 2: 0, 3: 1} -10.0
{0: 0, 1: 1, 2: 1, 3: 0} -9.0
{0: 0, 1: 1, 2: 0, 3: 1} -9.0
{0: 0, 1: 0, 2: 1, 3: 0} -8.0
{0: 0, 1: 0, 2: 0, 3: 1} -6.0
{0: 1, 1: 0, 2: 0, 3: 0} -5.0
{0: 1, 1: 0, 2: 1, 3: 0} -5.0
{0: 0, 1: 1, 2: 1, 3: 1} -5.0
{0: 1, 1: 1, 2: 0, 3: 0} -4.0
{0: 0, 1: 0, 2: 1, 3: 1} -4.0
{0: 0, 1: 1, 2: 0, 3: 0} -3.0
{0: 1, 1: 1, 2: 1, 3: 0} -2.0
{0: 1, 1: 0, 2: 1, 3: 1} -1.0
{0: 0, 1: 0, 2: 0, 3: 0} 0.0
{0: 1, 1: 1, 2: 1, 3: 1} 2.0


### 2. K4 Compete Graph

![img](https://mathworld.wolfram.com/images/eps-gif/CompleteGraphs_801.gif)

Solving problems with large numbers of variables might necessitate the use of decomposition methods such as branch-and-bound to reduce the number of variables. The following illustrative example reduces an Ising model for a small problem (the K4 complete graph), and converts the reduced-variables model to QUBO formulation.

In [14]:
import dimod
linear = {1: 1, 2: 2, 3: 3, 4: 4}
quadratic = {(1, 2): 12,
             (1, 3): 13,
             (1, 4): 14,
             (2, 3): 23,
             (2, 4): 24,
             (3, 4): 34}
offset = 0.0
vartype = dimod.SPIN
bqm_k4 = dimod.BinaryQuadraticModel(linear, quadratic, offset, vartype)

In [49]:
bqm_k4.info = {'Complete K4 binary quadratic model.'}
bqm_k4.info.issubset({'Complete K3 binary quadratic model.',
                     'Complete K4 binary quadratic model.',
                     'Complete K5 binary quadratic model.'})

True

In [50]:
bqm_k4.adj[2]

<ShapeableNeighbour: {1: 12, 3: 23, 4: 24}>

In [51]:
bqm_k4.adj[2][3]

23

In [52]:
bqm_k4.vartype

<Vartype.SPIN: frozenset({1, -1})>

In [53]:
len(bqm_k4.linear)

4

In [54]:
response = dimod.ExactSolver().sample(bqm_k4)

In [29]:
for sample, energy in response.data(['sample', 'energy']):
    print(sample, energy)
    break

{0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 1, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 1, 15: 0, 16: 0} -72630.0


In [21]:
bqm_k4.contract_variables(2, 3)
len(bqm_k4.linear)

3

In [22]:
bqm_no3_qubo = bqm_k4.binary

In [23]:
bqm_no3_qubo.vartype

<Vartype.BINARY: frozenset({0, 1})>

### 3. Number Partition Problem

In number theory and computer science, the partition problem, or number partitioning, is the task of deciding whether a given multiset S of positive integers can be partitioned into two subsets S1 and S2 such that the sum of the numbers in S1 equals the sum of the numbers in S2. Although the partition problem is NP-complete, there is a pseudo-polynomial time dynamic programming solution, and there are heuristics that solve the problem in many instances, either optimally or approximately. For this reason, it has been called "the easiest hard problem".

There is an optimization version of the partition problem, which is to partition the multiset S into two subsets S1, S2 such that the difference between the sum of elements in S1 and the sum of elements in S2 is minimized. The optimization version is NP-hard, but can be solved efficiently in practice

In [31]:
S =[25, 7,13, 31,23,12,45,45,67,24,34,56,67,42,17, 21,10]
len(S)

17

In [23]:
c = sum(S)
c 

539

In [24]:
Q = np.zeros([len(S),len(S)])

In [25]:
for i in range(len(S)):
    for j in range(len(S)):
        if i ==j:
            Q[i,i] = S[i]*(S[i] -c)
        else:
            Q[i,j] = S[i]*S[j]

In [26]:
Q

array([[-12850.,    175.,    325.,    775.,    575.,    300.,   1125.,
          1125.,   1675.,    600.,    850.,   1400.,   1675.,   1050.,
           425.,    525.,    250.],
       [   175.,  -3724.,     91.,    217.,    161.,     84.,    315.,
           315.,    469.,    168.,    238.,    392.,    469.,    294.,
           119.,    147.,     70.],
       [   325.,     91.,  -6838.,    403.,    299.,    156.,    585.,
           585.,    871.,    312.,    442.,    728.,    871.,    546.,
           221.,    273.,    130.],
       [   775.,    217.,    403., -15748.,    713.,    372.,   1395.,
          1395.,   2077.,    744.,   1054.,   1736.,   2077.,   1302.,
           527.,    651.,    310.],
       [   575.,    161.,    299.,    713., -11868.,    276.,   1035.,
          1035.,   1541.,    552.,    782.,   1288.,   1541.,    966.,
           391.,    483.,    230.],
       [   300.,     84.,    156.,    372.,    276.,  -6324.,    540.,
           540.,    804.,    288.,    4

In [27]:
vartype = dimod.BINARY
npp = dimod.BinaryQuadraticModel.from_numpy_matrix(Q,\
                    variable_order=None, offset=0.0, interactions=None)

In [ ]:
#S =[25, 7,13, 31, 42,17, 21,10]

In [30]:
response = dimod.ExactSolver().sample(npp)
for sample, energy in response.data(['sample', 'energy']):
    print(sample, energy)
    break

{0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 1, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 1, 15: 0, 16: 0} -72630.0


------------